# Modelagem Multinível do Efeito das Emendas PIX

Este notebook implementa a estratégia **multilevel step-up** descrita no TCC.
As etapas são:
1. **Modelo nulo**: estima a variância intra-partidos sem variáveis explanatórias.
2. **Modelo com interceptos aleatórios**: inclui `emendas_pix_per_capita_partido_prefeito_eleito` como efeito fixo, permitindo interceptos diferentes por partido.
3. **Modelo com interceptos e inclinações aleatórios**: além do intercepto, o efeito das emendas por habitante varia entre partidos.
4. **Modelo completo**: adiciona as variáveis *dummy* dos clusters socioeconômicos, controlando perfis municipais.


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf


In [ ]:
# Carrega a base unificada com dummies de clusters
base = pd.read_csv('../data/dados_com_clusters.csv')

# Converte as colunas de cluster para inteiro (0/1)
for col in ['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3']:
    base[col] = base[col].fillna(False).astype(int)


In [ ]:
# Modelo nulo
null_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ 1',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
null_res = null_model.fit()
print(null_res.summary())


In [ ]:
# Modelo com interceptos aleatórios
ri_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
ri_res = ri_model.fit()
print(ri_res.summary())


In [ ]:
# Modelo com interceptos e inclinações aleatórios
rs_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_per_capita_partido_prefeito_eleito',
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
rs_res = rs_model.fit()
print(rs_res.summary())


In [ ]:
# Modelo multinível completo com dummies dos clusters
full_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + cluster_1 + cluster_2 + cluster_3',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_per_capita_partido_prefeito_eleito',
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
full_res = full_model.fit()
print(full_res.summary())


## Análise dos coeficientes
A seguir resumimos os parâmetros do modelo completo para verificar quais variáveis apresentam efeitos estatisticamente significativos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
summary_df = pd.DataFrame({
    'coeficiente': full_res.params,
    'erro_padrao': full_res.bse,
    'p_valor': full_res.pvalues
})
summary_df

In [ ]:
coef_df = summary_df.loc[['emendas_pix_per_capita_partido_prefeito_eleito', 'cluster_1', 'cluster_2', 'cluster_3']]
coef_df = coef_df.reset_index().rename(columns={'index': 'variavel'})
plt.figure(figsize=(6, 4))
sns.pointplot(data=coef_df, x='coeficiente', y='variavel', join=False, xerr=1.96*coef_df['erro_padrao'])
plt.axvline(0, color='red', linestyle='--', linewidth=1)
plt.title('Coeficientes do modelo completo (95% IC)')
plt.xlabel('Efeito estimado')
plt.ylabel('')
plt.tight_layout()
plt.show()


Os coeficientes dos clusters 2 e 3 apresentam p-valores menores que 0.05, indicando que essas variáveis possuem efeito significativo. Em contrapartida, o coeficiente associado ao volume de Emendas PIX por prefeito não difere de zero no modelo final.